# 0. Installing Transformers and Importing Dependencies

In [ ]:
!pip install transformers

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [2]:
summarizer = pipeline("summarization")

# 2. Get Blog Post from Medium

In [3]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [4]:
r = requests.get(URL)

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [6]:
ARTICLE

'Will The Game Stop with Gamestop Or Is This Just The Beginning? Crypto, Markets, Trading The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business. The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers. One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time making money in markets

# 3. Chunk Text

In [7]:
max_chunk = 500

In [8]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [9]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [10]:
len(chunks)

8

# 4. Summarize Text

In [11]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [12]:
res[0]

{'summary_text': ' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers .'}

In [13]:
' '.join([summ['summary_text'] for summ in res])

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers .  Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’. The market here is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing. The fundamentals no longer matter; and this was demonstrated only 5 years later with what could be the largest bubble of viable assets in history .  Greenspans moral hazard of late 1994/1995 fundamentally changed the market in a new way . Extraordinary people who were never wrong in risk-adjusted terms and who u

In [89]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [90]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)